<a href="https://colab.research.google.com/github/GreeshmaHarids/puthon_class/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM , Dense
from keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
df=pd.read_csv('/content/GOOG.csv')
df.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-08-08,119.120003,120.860001,117.830002,118.139999,118.139999,17061100
1,2022-08-09,117.989998,118.199997,116.559998,117.500000,117.500000,15424300
2,2022-08-10,119.589996,121.779999,119.360001,120.650002,120.650002,20497000
3,2022-08-11,122.080002,122.339996,119.550003,119.820000,119.820000,16671600
4,2022-08-12,121.160004,122.650002,120.400002,122.650002,122.650002,16121100
5,2022-08-15,122.209999,123.260002,121.570000,122.879997,122.879997,15525000
6,2022-08-16,122.320000,123.227997,121.535004,122.510002,122.510002,15626200
7,2022-08-17,120.930000,122.150002,120.199997,120.320000,120.320000,17589200
8,2022-08-18,120.230003,121.690002,119.550003,120.860001,120.860001,15652000
9,2022-08-19,119.870003,120.000000,117.669998,118.120003,118.120003,20187000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       250 non-null    object 
 1   Open       250 non-null    float64
 2   High       250 non-null    float64
 3   Low        250 non-null    float64
 4   Close      250 non-null    float64
 5   Adj Close  250 non-null    float64
 6   Volume     250 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 13.8+ KB


In [ ]:
df['Date']=pd.to_datetime(df['Date'])
df.set_index(df['Date'],inplace=True)

In [ ]:
df.drop(columns=['Open','High','Low','Adj Close','Volume'],inplace=True)

In [ ]:
import plotly.graph_objects as go

In [ ]:
trace1=go.Scatter(
    x=df['Date'],
    y= df['Close'],
    mode='lines',
    name='Data'
)

layout=go.Layout(
    title='Google Stock',
    xaxis={'title':"Date"},
    yaxis={'title':"Close"}
)

fig=go.Figure(data=[trace1],layout=layout)
fig.show()

In [ ]:
Close_data=df['Close'].values
Close_data

array([118.139999, 117.5     , 120.650002, 119.82    , 122.650002,
       122.879997, 122.510002, 120.32    , 120.860001, 118.120003,
       115.07    , 114.769997, 114.699997, 117.699997, 111.300003,
       110.339996, 109.910004, 109.150002, 110.550003, 108.68    ,
       107.480003, 110.480003, 109.419998, 111.779999, 111.870003,
       105.309998, 105.870003, 103.900002, 103.629997, 103.849998,
       101.830002, 100.010002, 100.57    ,  99.169998,  98.809998,
        98.089996, 100.739998,  98.089996,  96.150002,  99.300003,
       102.410004, 102.220001, 102.239998,  99.57    ,  98.709999,
        98.050003,  98.300003,  99.709999,  97.18    , 100.779999,
       101.389999, 100.290001, 100.529999, 101.480003, 102.970001,
       104.93    ,  94.82    ,  92.599998,  96.580002,  94.660004,
        90.5     ,  87.07    ,  83.489998,  86.699997,  88.650002,
        88.910004,  87.400002,  94.169998,  96.730003,  96.029999,
        98.720001,  98.989998,  98.5     ,  97.800003,  95.830

In [ ]:
Close_data=Close_data.reshape(-1,1)
Close_data

array([[118.139999],
       [117.5     ],
       [120.650002],
       [119.82    ],
       [122.650002],
       [122.879997],
       [122.510002],
       [120.32    ],
       [120.860001],
       [118.120003],
       [115.07    ],
       [114.769997],
       [114.699997],
       [117.699997],
       [111.300003],
       [110.339996],
       [109.910004],
       [109.150002],
       [110.550003],
       [108.68    ],
       [107.480003],
       [110.480003],
       [109.419998],
       [111.779999],
       [111.870003],
       [105.309998],
       [105.870003],
       [103.900002],
       [103.629997],
       [103.849998],
       [101.830002],
       [100.010002],
       [100.57    ],
       [ 99.169998],
       [ 98.809998],
       [ 98.089996],
       [100.739998],
       [ 98.089996],
       [ 96.150002],
       [ 99.300003],
       [102.410004],
       [102.220001],
       [102.239998],
       [ 99.57    ],
       [ 98.709999],
       [ 98.050003],
       [ 98.300003],
       [ 99.7

In [ ]:
len(Close_data)

250

In [ ]:
split_percentage=0.80
split=int(split_percentage*len(Close_data))

In [ ]:
split

200

In [ ]:
#it is converted since it is passed to model
close_train=Close_data[:split]
close_test=Close_data[split:]

In [ ]:
len(close_train)

200

In [ ]:
len(close_test)

50

In [ ]:
date_train=df['Date'][:split]
date_test=df['Date'][split:]

In [ ]:
look_back=15 #number of time steps to look back

train_generator= TimeseriesGenerator(close_train,close_train,length=look_back,batch_size=20)
tets_generator= TimeseriesGenerator(close_test,close_test,length=look_back,batch_size=1)


In [ ]:
model=Sequential()

model.add(LSTM(10,activation='relu',input_shape=(look_back,1)))

model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam',loss='mse')

In [ ]:
#training
model.fit_generator(train_generator,epochs=200,verbose=1)

Epoch 1/200
10/10 [==============================] - 0s 7ms/step - loss: 11485.8770
Epoch 2/200
 1/10 [==>...........................] - ETA: 0s - loss: 12442.0918

<ipython-input-36-ce576b5d087c>:2: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



10/10 [==============================] - 0s 7ms/step - loss: 11337.5908
Epoch 3/200
10/10 [==============================] - 0s 6ms/step - loss: 11202.8691
Epoch 4/200
10/10 [==============================] - 0s 6ms/step - loss: 11063.1543
Epoch 5/200
10/10 [==============================] - 0s 7ms/step - loss: 10929.7754
Epoch 6/200
10/10 [==============================] - 0s 6ms/step - loss: 10801.8828
Epoch 7/200
10/10 [==============================] - 0s 6ms/step - loss: 10677.0547
Epoch 8/200
10/10 [==============================] - 0s 6ms/step - loss: 10553.8154
Epoch 9/200
10/10 [==============================] - 0s 6ms/step - loss: 10433.2734
Epoch 10/200
10/10 [==============================] - 0s 6ms/step - loss: 10316.0459
Epoch 11/200
10/10 [==============================] - 0s 6ms/step - loss: 10200.5547
Epoch 12/200
10/10 [==============================] - 0s 8ms/step - loss: 10078.2285
Epoch 13/200
10/10 [==============================] - 0s 7ms/step - loss: 9959.8779
E

In [ ]:
prediction=model.predict_generator(tets_generator)

<ipython-input-38-85fa9324d7c3>:1: UserWarning:

`Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.



In [ ]:
close_train=close_train.reshape((-1))
close_test=close_test.reshape((-1))
prediction=prediction.reshape((-1))

In [ ]:
prediction

array([127.53497 , 128.8141  , 126.95016 , 127.50946 , 124.53829 ,
       128.22401 , 126.02733 , 122.08297 , 124.659195, 125.68732 ,
       123.4781  , 125.20731 , 124.110146, 126.05481 , 123.57415 ,
       123.57724 , 119.80261 , 122.6486  , 123.24057 , 126.156815,
       126.10523 , 125.87824 , 125.64701 , 124.78993 , 122.06565 ,
       124.04309 , 124.66962 , 124.9176  , 128.94513 , 129.27246 ,
       131.16692 , 131.53525 , 131.4732  , 129.93877 , 130.58456 ],
      dtype=float32)

In [ ]:
trace1=go.Scatter(
    x=date_train,
    y=close_train,
    mode='lines',
    name='data'
)

trace2=go.Scatter(
    x=date_test,
    y=close_test,
    mode='lines',
    name='test'
)
trace3=go.Scatter(
    x=date_test,
    y=prediction,
    mode='lines',
    name='prediction'
)

layout=go.Layout(
    title='Google stock prediction',
    xaxis={'title':"Date"},
    yaxis={'title':"Close"}
)
fig=go.Figure(data=[trace1,trace2,trace3],layout=layout)
fig.show()